In [1]:
import pandas as pd
import io
import requests
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20230405-154703/valeursfoncieres-2018.txt"
s = requests.get(url).content
c = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')

C:\Users\dargo\AppData\Local\Temp\ipykernel_20916\1361649060.py:3: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  c = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')


# Cleaning

In [3]:
c.drop(columns=['Identifiant de document',
                'Reference document',
                '1 Articles CGI',
                '2 Articles CGI',
                '3 Articles CGI',
                '4 Articles CGI',
                '5 Articles CGI',
                'Code commune',
                'Prefixe de section',
                'Section',
                'No plan',
                'No Volume',
                '1er lot',
                'Surface Carrez du 1er lot',
                '2eme lot',
                'Surface Carrez du 2eme lot',
                '3eme lot',
                'Surface Carrez du 3eme lot',
                '4eme lot',
                'Surface Carrez du 4eme lot',
                '5eme lot',
                'Surface Carrez du 5eme lot',
                'Identifiant local',
                'Type local'
                ], axis=1, inplace=True)

In [4]:
c.dropna(subset=['Valeur fonciere'], inplace=True)

In [5]:
c['No voie'].fillna(0, inplace=True)

In [6]:
c['Code postal'].fillna(0, inplace=True)

In [7]:
c['Code type local'].fillna(0, inplace=True)

In [8]:
c['Surface reelle bati'].fillna(0, inplace=True)

In [9]:
c['Nombre pieces principales'].fillna(0, inplace=True)

In [10]:
c['Surface terrain'].fillna(0, inplace=True)

In [11]:
c['B/T/Q'].fillna('0', inplace=True)
BTQ = preprocessing.LabelEncoder()
BTQ.fit(c['B/T/Q'])
c['B/T/Q'] =  BTQ.transform(c['B/T/Q'])

In [12]:
c['Type de voie'].fillna('', inplace=True)
Type_Voie = preprocessing.LabelEncoder()
Type_Voie.fit(c['Type de voie'])
c['Type de voie'] =  Type_Voie.transform(c['Type de voie'])

In [13]:
c['Code voie'] = c['Code voie'].astype(str)
c['Code voie'].fillna('', inplace=True)
Code_Voie = preprocessing.LabelEncoder()
Code_Voie.fit(c['Code voie'])
c['Code voie'] =  Code_Voie.transform(c['Code voie'])

In [14]:
c['Voie'].fillna('', inplace=True)
Voie = preprocessing.LabelEncoder()
Voie.fit(c['Voie'])
c['Voie'] =  Voie.transform(c['Voie'])

In [15]:
c['Nature culture'].fillna('', inplace=True)
NC = preprocessing.LabelEncoder()
NC.fit(c['Nature culture'])
c['Nature culture'] =  NC.transform(c['Nature culture'])

In [16]:
c['Nature culture speciale'].fillna('', inplace=True)
NCS = preprocessing.LabelEncoder()
NCS.fit(c['Nature culture speciale'])
c['Nature culture speciale'] =  NCS.transform(c['Nature culture speciale'])

In [18]:
mutation_values = c['Nature mutation'].drop_duplicates().values
for idx, i in enumerate(mutation_values) :
    c['Nature mutation'] = c['Nature mutation'].str.replace(i, str(idx))
    
c['Nature mutation']= c['Nature mutation'].apply(int)

In [19]:
c['Date mutation'] = pd.to_datetime(c['Date mutation'], format='mixed')
c['Date mutation'] = c['Date mutation'].apply(lambda x : x.timestamp())

In [20]:
c.dtypes

No disposition                 int64
Date mutation                float64
Nature mutation                int64
Valeur fonciere               object
No voie                      float64
B/T/Q                          int32
Type de voie                   int32
Code voie                      int32
Voie                           int32
Code postal                  float64
Commune                       object
Code departement              object
Nombre de lots                 int64
Code type local              float64
Surface reelle bati          float64
Nombre pieces principales    float64
Nature culture                 int32
Nature culture speciale        int32
Surface terrain              float64
dtype: object

# Y columns

In [21]:
X = c.drop(['Valeur fonciere'], axis=1)
Y = c['Valeur fonciere'].str.replace(',', '.')
Y = Y.astype('float64')
Y = Y.astype(int)

In [22]:
Y_10 = Y / 10
Y_10 = Y_10.astype(int)
Y_100 = Y_10 / 10
Y_100 = Y_100.astype(int)
Y_1000 = Y_100 / 10
Y_1000 = Y_1000.astype(int)
Y_10000 = Y_1000 / 10
Y_10000 = Y_10000.astype(int)

# Division

In [23]:
X_test, X_train, Y_test, Y_train = train_test_split(X, Y, train_size=0.7, random_state=42)

In [24]:
X_test_10, X_train_10, Y_test_10, Y_train_10 = train_test_split(X, Y_10, train_size=0.7, random_state=42)

In [25]:
X_test_100, X_train_100, Y_test_100, Y_train_100 = train_test_split(X, Y_100, train_size=0.7, random_state=42)

In [26]:
X_test_1000, X_train_1000, Y_test_1000, Y_train_1000 = train_test_split(X, Y_1000, train_size=0.7, random_state=42)

# Model training

## Random forest

In [27]:
clf = RandomForestClassifier(n_estimators=100,max_depth=5, random_state=0)
clf.fit(X_train_1000,Y_train_1000)

ValueError: could not convert string to float: 'LE GLAIZIL'

In [ ]:
 clf.predict_proba(X_test_1000)  